<a href="https://colab.research.google.com/github/fatemehra10/predict-weather/blob/main/forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# import necessary library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Preprocessing

In [78]:
#Read data
weather = pd.read_csv('weather.csv')
weather.head()

,time,date,city,temp,wind,wind_direction,humidity,barometer,visibility,weather
0,00:30,January 2023,Abadan,23 °C,7 km/h,Wind blowing from 110° East-southeast to West-...,89%,1016 mbar,6 km,Passing clouds.
1,01:30,January 2023,Abadan,22 °C,4 km/h,Wind blowing from 150° South-southeast to Nort...,94%,1016 mbar,6 km,Clear.
2,02:30,January 2023,Abadan,21 °C,4 km/h,Wind blowing from 40° Northeast to Southwest,94%,1016 mbar,6 km,Clear.
3,04:30,January 2023,Abadan,20 °C,4 km/h,Wind blowing from 340° North-northwest to Sout...,94%,1017 mbar,4 km,Fog.
4,05:30,January 2023,Abadan,20 °C,4 km/h,Wind blowing from 80° East to West,100%,1017 mbar,4 km,Fog.


In [79]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40066 entries, 0 to 40065
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   time            40066 non-null  object
 1   date            40066 non-null  object
 2   city            40066 non-null  object
 3   temp            40066 non-null  object
 4   wind            40061 non-null  object
 5   wind_direction  40066 non-null  object
 6   humidity        40065 non-null  object
 7   barometer       40062 non-null  object
 8   visibility      28073 non-null  object
 9   weather         40066 non-null  object
dtypes: object(10)
memory usage: 3.1+ MB


In [80]:
# Cleaning data
weather.temp = weather.temp.apply(lambda x: str(x).replace('°C',''))
weather.wind = weather.wind.apply(lambda x: str(x).replace('km/h','').replace('No wind','-1'))
weather.humidity = weather.humidity.apply(lambda x: str(x).replace('%',''))
weather.barometer = weather.barometer.apply(lambda x: str(x).replace('mbar',''))
weather.visibility = weather.visibility.apply(lambda x: str(x).replace('km',''))
weather.weather = weather.weather.apply(lambda x: str(x).replace('.',''))

In [81]:
# Convert datatypes
weather.time = weather.time.astype('str')
weather.date = weather.date.astype('str')
weather.city = weather.city.astype('category')
weather.temp = weather.temp.astype(int)
weather.wind = weather.wind.astype(float)
weather.wind_direction = weather.wind_direction.astype('category')
weather.humidity = weather.humidity.astype(float)
weather.barometer = weather.barometer.astype(float)
weather.visibility = weather.visibility.astype(float)
weather.weather = weather.weather.astype('category')

In [82]:
# Rename column name 
weather.rename(columns = {'temp':'temp(°C)','wind':'wind(km/h)','humidity':'humidity(%)','barometer':
                          'barometer(mbar)','visibility':'visibility(km)'},inplace = True)

In [83]:
# Handle NAN
weather['visibility(km)'].fillna(round(weather['visibility(km)'].mean(),2),inplace = True)
weather.dropna(inplace = True)

In [84]:
weather.to_csv('clean_weather.csv',index = False)

In [91]:
dict_weather = {
    'Partly sunny':'Sunny',
    'Passing clouds' : 'Overcast',                           
    'Scattered clouds' : 'Overcast',
    'Broken clouds' :'Overcast',
    'Partly cloudy' : 'Overcast',
    'More clouds than sun': 'Overcast',
    'Mostly cloudy':'Overcast',
    'Light rain Mostly cloudy':'Rain-Overcast',
    'Ice fog' :'Haze',
    'Rain Mostly cloudy':'Rain-Overcast',
    'Light rain More clouds than sun': 'Rain-Overcast',
    'Light rain Overcast': 'Rain-Overcast',
    'Rain Fog' : 'Drizzle Haze',
    'Thunderstorms Passing clouds' : 'Thunderstorms-Overcast',
    'Light rain Fog' : 'Drizzle Haze',
    'Thunderstorms Mostly cloudy':'Thunderstorms-Overcast',
    'Thunderstorms Broken clouds':'Thunderstorms-Overcast',
    'Low clouds':'Overcast',
    'Thunderstorms Scattered clouds':'Thunderstorms-Overcast',
    'Rain More clouds than sun':'Rain-Overcast',
    'Thundershowers Passing clouds':'Thunderstorms-Overcast',
    'Drizzle Mostly cloudy':'Rain-Overcast',
    'Thunderstorms More clouds than sun':'Thunderstorms-Overcast',
    'Sprinkles More clouds than sun':'Overcast',
    'Rain Overcast':'Rain-Overcast',
    'Thunderstorms Partly cloudy':'Thunderstorms-Overcast',
    'Sprinkles Broken clouds':'Overcast',
    'Scattered showers Broken clouds':'Overcast',
    'Light snow Mostly cloudy':'Snow-Overcast',
    'Cloudy':'Overcast',
    'Heavy rain Mostly cloudy':'Rain-Overcast',
    'Sprinkles Passing clouds':'Overcast',
    'Mild':'Clear',
    'Light rain Cloudy':'Rain-Overcast',
    'Sprinkles Partly cloudy':'Overcast',
    'Sprinkles Mostly cloudy':'Overcast',
    'Light rain Partly cloudy':'Rain-Overcast',
    'Dense fog':'Haze',
    'Light snow Fog':'Snow-Haze',
    'Sprinkles Fog':'Haze',
    'Scattered showers Scattered clouds':'Rain-Overcast',
    'Sprinkles Scattered clouds':'Overcast',     
    'Light rain Passing clouds':'Rain-Overcast',
    'Thunderstorms Partly sunny':'Thunderstorms-Sunny',
    'Light snow Overcast':'Snow-Overcast',
    'Rain showers More clouds than sun':'Rain-Overcast',
    'Low level haze':'Haze',
    'Sprinkles Overcast':'Overcast',
    'Thunderstorms Overcast': 'Thunderstorms-Overcast',
    'Rain Cloudy':'Rain-Overcast',
    'Sprinkles Partly sunny':'Sunny',
    'Drizzle Overcast' : 'Rain-Overcast',
    'Snow Overcast':'Snow-Overcast',
    'Thundershowers Partly cloudy': 'Thunderstorms-Overcast',
    'Thundershowers Scattered clouds': 'Thunderstorms-Overcast',
    'Drizzle Low clouds':'Rain-Overcast',
    'Pleasantly warm':'Sunny',                          
    'Light snow Ice fog':'Snow-Haze',
    'Rain showers Broken clouds':'Rain-Overcast',
    'Light rain Broken clouds' :'Rain-Overcast',
    'Scattered showers More clouds than sun':'Rain-Overcast',
    'Chilly':'Cold',
    'Thundershowers Overcast':'Thunderstorms-Overcast',
    'Light snow Cloudy':'Sonw-Overcast',
    'Light freezing rain Ice fog':'Rain-Haze',
    'Thunderstorms Fog': 'Thunderstorms-Haze',
    'Rain showers Mostly cloudy':'Rain-Overcast',
    'Scattered showers Passing clouds':'Rain-Overcast',
    'Snow Ice fog':'Snow-Haze',
    'Scattered showers Partly sunny':'Rain',
    'Strong thunderstorms Mostly cloudy':'Thunderstorms-Overcast',
    'Hot':'Sunny',
    'Fog':'Haze',
    'Heavy rain Cloudy':'Rain-Overcast',
    'Heavy snow Dense fog':'Snow-Haze',
    'Rain showers Partly cloudy':'Rain-Overcast',
    'Snow Mostly cloudy':'Snow-Overcast',
    'Snow Fog':'Snow-Haze',
    'Snow More clouds than sun':'Snow-Overcast',
    'Rain showers Fog':'Rain-Haze',
    'Rain showers Overcast':'Rain-Overcast',
    'Sprinkles Cloudy':'Overcast',
    'Rain Broken clouds':'Rain-Overcast',
    'Strong thunderstorms Cloudy':'Thunderstorms-Overcast',
    'Light fog':'Haze',
    'Thunderstorms Cloudy':'Thunderstorms-Overcast',
    'Scattered showers Overcast':'Rain-Overcast',
    'Rain Partly cloudy':'Rain-Overcast',
    'Light rain Scattered clouds':'Rain-Overcast',
    'Drizzle Broken clouds':'Rain-Overcast',
    'Drizzle Cloudy':'Rain-Overcast',
    'Light rain Partly sunny':'Rain',
    'Thundershowers Broken clouds':'Thunderstorms-Overcast',
    'Light snow More clouds than sun':'Snow-Overcast',
    'Heavy rain Fog':'Rain-Haze',
    'Strong thunderstorms Broken clouds':'Thunderstorms-Overcast',
    'Snow showers Broken clouds':'Snow-Overcast',
    'Snow flurries Overcast' :'Snow-Overcast',
    'Snow flurries More clouds than sun':'Snow-Overcast',
    'Sprinkles Sandstorm':'Sandstorm',
    'Rain showers Passing clouds':'Rain-Overcast',
    'Mixture of precip Ice fog':'Haze',
    'Refreshingly cool':'Cold',
    'Mixture of precip Fog':'Haze',
    'Light snow Passing clouds':'Snow-Overcast',
    'Extremely hot':'Sunny',
    'Drizzle Partly sunny':'Rain',
    'Rain Low clouds':'Rain-Overcast',
    'Light snow Partly cloudy':'Snow-Overcast',
    'Light snow Broken clouds':'Snow-Overcast',
    'Rain showers Partly sunny':'Rain-Overcast',
    'Heavy rain More clouds than sun':'Rain',
    'Scattered showers Fog':'Haze',
    'Heavy snow Cloudy':'Snow-Overcast',
    'Scattered showers Partly cloudy':'Overcast',
    'Light rain Ice fog':'Rain-Haze',
    'Light mixture of precip Overcast':'Rain-Overcast',
    'Thundershowers Mostly cloudy':'Thunderstorms-Overcast',
    'Heavy rain Overcast':'Rain-Overcast',
    'Light snow Low clouds':'Snow-Overcast',
    'Cool':'Cold',
    'Light rain Low clouds':'Rain-Overcast',
    'Thundershowers Partly sunny':'Thunderstorms-Sunny',
    'Heavy snow Fog':'Snow-Haze',
    'Lots of rain Fog':'Rain-Haze',
    'Light rain Sandstorm':'Rain',
    'Snow Broken clouds':'Snow-Overcast',
    'Lots of rain Partly sunny':'Rain',
    'Strong thunderstorms Partly cloudy':'Thunderstorms-Overcast',
    'Snow flurries Cloudy':'Snow-Overcast',
    'Hail Mostly cloudy':'Rain-Overcast',
    'Rain Passing clouds':'Rain-Overcast',
    'Rain Scattered clouds':'Rain-Overcast',
    'Rain showers Cloudy':'Rain-Overcast',
    'Heavy snow Overcast':'Snow-Overcast',
    'Scattered showers Mostly cloudy':'Rain-Overcast',
    'Snow showers Overcast':'Snow-Overcast',
    'Snow flurries Scattered clouds':'Snow-Overcast',
    'Snow flurries Passing clouds':'Snow-Overcast',
    'Snow flurries Partly cloudy':'Snow-Overcast',
    'Sleet Haze':'Haze',
    'Sleet Mostly cloudy':'Snow-Overcast',
    'Heavy snow Ice fog':'Snow-Haze',
    'Drizzle Haze':'Rain-Haze',
    'Drizzle Fog':'Rain-Haze',
    'Sandstorm':'Duststorm'

}



In [92]:
def clean_target(x):
  if x in dict_weather.keys():
    return dict_weather[x]
  else:
    return x

weather.weather = weather.weather.apply(clean_target)

In [93]:
weather.weather.value_counts()

Overcast                  14555
Clear                     10683
Sunny                     10176
Haze                       3080
Rain-Overcast               731
Thunderstorms-Overcast      309
Duststorm                   179
Rain-Haze                   165
Snow-Overcast                78
Snow-Haze                    43
Cold                         16
Rain                         15
Thunderstorms-Sunny          13
Thunderstorms-Haze            7
Sonw-Overcast                 7
Name: weather, dtype: int64

# Exploratory Analysis

# Reduction dimension

# Classification